__BY SDV (install)__
https://docs.sdv.dev/sdv/installation

criar nvo ambiente:

conda install numpy=1.21 pandas=1.3 -y

usar a versão para anaconda

In [1]:
import torch
import pandas as pd
import sdv
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sklearn.preprocessing import MinMaxScaler

In [2]:
# CTGAN precisa dos dados com features - normalizados
data_path = "../../data/features/data_train_features.csv"
data = pd.read_csv(data_path)

print("Data shape", data.shape)

Data shape (330984, 27)


In [3]:
data.head()

,mean_x-accel,peak_x-accel,abs_dev_x-accel,std_x-accel,mean_y-accel,peak_y-accel,abs_dev_y-accel,std_y-accel,mean_z-accel,peak_z-accel,...,peak_y-gyro,abs_dev_y-gyro,std_y-gyro,mean_z-gyro,peak_z-gyro,abs_dev_z-gyro,std_z-gyro,resultant_accel,resultant_gyro,activity
0,7.019093,35.508982,0.029864,0.037623,1.253040,40.821918,0.081643,0.108398,6.930752,33.908046,...,31.481481,0.017265,0.021700,-0.006869,31.322751,0.013767,0.017877,7.019093,0.021392,1
1,7.075111,38.076923,0.053842,0.086880,2.528753,49.256198,0.101113,0.129301,6.577109,39.200000,...,33.502825,0.018807,0.024530,0.001009,30.408163,0.018829,0.023044,7.075111,0.024693,1
2,7.070889,38.516129,0.053464,0.065662,2.322509,40.833333,0.126651,0.153066,6.547439,32.994350,...,31.648936,0.018747,0.024351,-0.009966,31.041667,0.014771,0.020038,7.070889,0.027268,1
3,6.124571,33.055556,0.061586,0.072122,4.951734,36.728395,0.056678,0.072093,5.930814,34.651163,...,30.203046,0.019995,0.025127,-0.007154,31.764706,0.013203,0.016663,6.124571,0.021019,1
4,7.047367,34.022989,0.036206,0.045813,1.199406,41.111111,0.073270,0.094186,6.911336,33.908046,...,30.410256,0.016114,0.020451,-0.006852,32.747253,0.013447,0.016822,7.047367,0.019003,1


In [10]:
scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(data)

scaled_data_df = pd.DataFrame(scaled_data, columns=data.columns)
scaled_data_df.head(2)

,mean_x-accel,peak_x-accel,abs_dev_x-accel,std_x-accel,mean_y-accel,peak_y-accel,abs_dev_y-accel,std_y-accel,mean_z-accel,peak_z-accel,...,peak_y-gyro,abs_dev_y-gyro,std_y-gyro,mean_z-gyro,peak_z-gyro,abs_dev_z-gyro,std_z-gyro,resultant_accel,resultant_gyro,activity
0,0.937273,0.013483,0.001074,0.001103,0.305997,0.027808,0.003927,0.004372,0.861890,0.028761,...,0.010326,0.00112,0.001256,0.543852,0.003741,0.000551,0.000674,0.472970,0.002190,0.0
1,0.939685,0.019710,0.003359,0.004983,0.377829,0.050457,0.005307,0.005583,0.841896,0.064918,...,0.018960,0.00178,0.002355,0.548364,0.002510,0.001769,0.001692,0.476763,0.003481,0.0


In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(scaled_data_df)
print(metadata)

In [ ]:
synthesizer = CTGANSynthesizer(
    metadata=metadata, 
    enforce_rounding=True, 
    epochs=10000, 
    verbose=True, 
    cuda=True
)

synthesizer.fit(scaled_data_df)
synthesizer.save(filepath="synthesizer_10000epochs.pkl")

syn_data = synthesizer.sample(num_rows=1000000)



loss_values = synthesizer.get_loss_values()
print(loss_values)

In [ ]:
# Inverse transform to get back original data
syn_data_descaled = scaler.inverse_transform(syn_data)

syn_data_df = pd.DataFrame(syn_data_descaled, columns=data.columns)

syn_data_df.to_csv("synthetic_data_1000000.csv")